In [1]:
!pip3 install turfpy
import pandas as pd
import os
import hashlib
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, Feature
import shutil

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
file_path="~/SageMaker/CDEM/CMLs_20110609_20110813.dat"

In [3]:
df = pd.read_table(file_path, sep="\s+", index_col=False)

In [4]:
df["Start_Concat"]=df["YStart"].astype(str)+" "+df["XStart"].astype(str)
df["End_Concat"]=df["YEnd"].astype(str)+" "+df["XEnd"].astype(str)

In [5]:
def convert_to_four_letter_hash(concat):
    h = hashlib.sha1(str.encode(concat))
    d = h.digest()
    s = ""
    for i in range(0,4):
        x = d[i] % 52
        if x >= 26:
            s += chr(ord('A') + x - 26)
        else:
            s += chr(ord('a') + x)
    return s

In [6]:
print(f'{convert_to_four_letter_hash("51.80485 5.11687")}-{convert_to_four_letter_hash("51.83246 5.08996")}')


XmQw-fTxW


In [7]:
def convert_datetime_to_time(datetime):
    yyyy=datetime[:4]
    mm=datetime[4:6]
    dd=datetime[6:8]
    hh=datetime[-4:-2]
    ss=datetime[-2:]
    
    return(f"{dd}-{mm}-{yyyy} {hh}:{ss}")
    

In [8]:
df["ID"]=df["Start_Concat"].apply(convert_to_four_letter_hash).astype(str)+"-"+df["End_Concat"].apply(convert_to_four_letter_hash).astype(str)

In [9]:
df["Time"]=df["DateTime"].astype(str).apply(convert_datetime_to_time)

In [10]:
ids=[x for x in df.ID.unique() if x]

In [11]:
df["PowerTLTMmax[dBm]"]=5
df["PowerTLTMmin[dBm]"]=4.8
df["PowerTLTMmax[dBm]_baseline"]=0
df["PowerTLTMmin[dBm]_baseline"]=0
df["PowerRLTMmax[dBm]_baseline"]=0
df["PowerRLTMmin[dBm]_baseline"]=0
df.rename(columns={"Pmin": "PowerRLTMmin[dBm]", "Pmax": "PowerRLTMmax[dBm]"}, inplace=True)
df.dropna(inplace=True)

In [12]:
df[df.ID=="XJHB-jwZI"]

,Frequency,DateTime,PowerRLTMmin[dBm],PowerRLTMmax[dBm],PathLength,XStart,YStart,XEnd,YEnd,ID,Start_Concat,End_Concat,Time,PowerTLTMmax[dBm],PowerTLTMmin[dBm],PowerTLTMmax[dBm]_baseline,PowerTLTMmin[dBm]_baseline,PowerRLTMmax[dBm]_baseline,PowerRLTMmin[dBm]_baseline
0,39.263,201106090815,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 08:15,5,4.8,0,0,0,0
5410,39.263,201106090830,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 08:30,5,4.8,0,0,0,0
8347,39.263,201106090845,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 08:45,5,4.8,0,0,0,0
9709,39.263,201106090900,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 09:00,5,4.8,0,0,0,0
12958,39.263,201106090915,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 09:15,5,4.8,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451707,39.263,201108081245,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 12:45,5,4.8,0,0,0,0
1454381,39.263,201108081300,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 13:00,5,4.8,0,0,0,0
1456428,39.263,201108081315,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 13:15,5,4.8,0,0,0,0
1457720,39.263,201108081330,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 13:30,5,4.8,0,0,0,0


In [23]:
root="~/SageMaker/CDEM/CellEnMon/datasets/dme/09062011_13082011/raw6"
columns=["Time","PowerTLTMmax[dBm]_baseline","PowerTLTMmin[dBm]_baseline","PowerRLTMmax[dBm]_baseline","PowerRLTMmin[dBm]_baseline"]

polygon = Polygon(
    [
        [
            (5.180,52.097), #260
            (5.145,51.857), #356
            (4.926,51.968), #348
            
        ]
    ]
)

count=0
for id in ids[:]:
    count+=1
    T,R=id.split("-")
    d=df[(df.ID==id) & (df.Frequency.astype(float)>30) & (df.Frequency.astype(float)<45)] # & ((df.Frequency).astype(float)<45)
    window_size = 15
    print(f"Loading:{count}/{len(ids)}")
    if not d.empty:
        YStart=float(d.YStart.iloc[0])
        YEnd=float(d.YEnd.iloc[0])
        YCenter=(YStart+YEnd)/2

        XStart=float(d.XStart.iloc[0])
        XEnd=float(d.XEnd.iloc[0])
        XCenter=(XStart+XEnd)/2
        point = Feature(geometry=Point((XCenter, YCenter)))
        print(f"center:{XCenter, YCenter}")

        if boolean_point_in_polygon(point, polygon):
            print(f"Processing {id}")
            for i in range(len(df) - window_size + 1): #
                if i%100000==0:
                    print(i)
                for c in ["PowerTLTMmax[dBm]", "PowerTLTMmin[dBm]", "PowerRLTMmax[dBm]", "PowerRLTMmin[dBm]"]:

                    window = d.iloc[i:i+window_size-1][c].values.tolist()
                    if len(window)==window_size-1:
                        middle_val = window[window_size//2]
                        if 'RLT' in c:
                            d[f"{c}_baseline"].iloc[i]=middle_val-min(window)
                        elif 'TLT' in c:
                            d[f"{c}_baseline"].iloc[i]=max(window)-middle_val



            d.to_csv(f'{root}/{T}_{d.YStart.iloc[0]}_{d.XStart.iloc[0]}_{R}_{d.YEnd.iloc[0]}_{d.XEnd.iloc[0]}.csv',index=False, columns=columns)
            print(f"Done Processing {id}")



Loading:1/3083
center:(4.773985, 52.400335)
Loading:2/3083
center:(5.060975, 52.507315)
Loading:3/3083
center:(4.968719999999999, 52.504735)
Loading:4/3083
center:(4.786185, 52.44319)
Loading:5/3083
center:(4.800575, 52.402525)
Loading:6/3083
center:(4.76457, 52.50139)
Loading:7/3083
center:(4.835285, 52.433665)
Loading:8/3083
center:(4.747294999999999, 52.45661)
Loading:9/3083
center:(4.9578050000000005, 52.503315)
Loading:10/3083
center:(5.060975, 52.507315)
Loading:11/3083
center:(5.04922, 52.50904)
Loading:12/3083
center:(4.647715, 52.38302)
Loading:13/3083
Loading:14/3083
center:(4.789215, 52.479884999999996)
Loading:15/3083
center:(4.588884999999999, 52.45589)
Loading:16/3083
center:(4.674305, 52.503945)
Loading:17/3083
center:(4.789194999999999, 52.398405)
Loading:18/3083
center:(5.07592, 52.457984999999994)
Loading:19/3083
Loading:20/3083
center:(5.07592, 52.457984999999994)
Loading:21/3083
center:(4.789194999999999, 52.398405)
Loading:22/3083
Loading:23/3083
Loading:24/3083
ce

Loading:200/3083
Loading:201/3083
center:(5.26136, 52.704445)
Loading:202/3083
center:(5.0647850000000005, 52.75683)
Loading:203/3083
center:(4.99249, 52.84364)
Loading:204/3083
Loading:205/3083
center:(4.75126, 52.942715)
Loading:206/3083
center:(5.276285, 52.701375)
Loading:207/3083
center:(4.76552, 52.95547)
Loading:208/3083
center:(5.02457, 52.846565)
Loading:209/3083
center:(4.76552, 52.95547)
Loading:210/3083
Loading:211/3083
center:(4.775235, 52.63506)
Loading:212/3083
center:(4.74324, 52.620785)
Loading:213/3083
center:(4.757035, 52.615315)
Loading:214/3083
center:(4.73414, 52.6105)
Loading:215/3083
center:(4.755815, 52.60677)
Loading:216/3083
center:(5.84087, 52.40828500000001)
Loading:217/3083
center:(5.816665, 52.377759999999995)
Loading:218/3083
Loading:219/3083
center:(5.246295, 52.384969999999996)
Loading:220/3083
center:(5.472384999999999, 52.525909999999996)
Loading:221/3083
Loading:222/3083
center:(5.866095, 52.40768)
Loading:223/3083
center:(5.504835, 52.510665)
Loadi

Loading:412/3083
center:(5.40664, 52.18427)
Loading:413/3083
center:(5.39523, 52.1549)
Loading:414/3083
center:(5.36298, 52.161075)
Loading:415/3083
center:(5.40664, 52.18427)
Loading:416/3083
center:(5.360355, 52.175795)
Loading:417/3083
center:(5.41228, 52.18759)
Loading:418/3083
center:(5.55717, 52.130539999999996)
Loading:419/3083
center:(5.320095, 52.210565)
Loading:420/3083
center:(5.296775, 52.187805)
Loading:421/3083
Loading:422/3083
center:(5.392335, 52.18343)
Loading:423/3083
Loading:424/3083
center:(5.35356, 52.204595)
Loading:425/3083
center:(5.296775, 52.187805)
Loading:426/3083
center:(5.477365, 52.20715)
Loading:427/3083
center:(5.581105000000001, 52.129165)
Loading:428/3083
center:(5.6125050000000005, 52.264880000000005)
Loading:429/3083
center:(5.477365, 52.20715)
Loading:430/3083
Loading:431/3083
Loading:432/3083
center:(5.4569399999999995, 52.182275000000004)
Loading:433/3083
center:(5.296105, 52.162195)
Loading:434/3083
center:(5.4569399999999995, 52.182275000000004

Loading:609/3083
center:(5.30808, 51.721104999999994)
Loading:610/3083
Loading:611/3083
center:(4.275865, 52.10556)
Loading:612/3083
center:(4.26452, 52.102064999999996)
Loading:613/3083
Loading:614/3083
center:(4.3188200000000005, 52.068555)
Loading:615/3083
center:(4.3323149999999995, 52.08193)
Loading:616/3083
center:(4.33831, 52.09763)
Loading:617/3083
center:(4.344390000000001, 52.07358)
Loading:618/3083
center:(4.290045, 52.112175)
Loading:619/3083
center:(4.28655, 52.11229)
Loading:620/3083
center:(4.283524999999999, 52.09729)
Loading:621/3083
center:(4.28129, 52.101535)
Loading:622/3083
Loading:623/3083
center:(4.32892, 52.068835)
Loading:624/3083
center:(4.336015, 52.090379999999996)
Loading:625/3083
center:(4.28623, 52.10573)
Loading:626/3083
center:(4.28623, 52.10573)
Loading:627/3083
Loading:628/3083
Loading:629/3083
Loading:630/3083
Loading:631/3083
center:(5.160220000000001, 52.26258)
Loading:632/3083
center:(6.696714999999999, 51.9611)
Loading:633/3083
center:(6.722155, 

Loading:793/3083
center:(6.771245, 52.25574)
Loading:794/3083
center:(6.766575, 52.25212500000001)
Loading:795/3083
center:(6.49068, 52.35966)
Loading:796/3083
center:(6.82, 52.275695)
Loading:797/3083
center:(6.4579249999999995, 52.378695)
Loading:798/3083
center:(6.750145, 52.23187)
Loading:799/3083
center:(6.761315, 52.29282)
Loading:800/3083
center:(6.781965, 52.253150000000005)
Loading:801/3083
center:(6.812585, 52.404465)
Loading:802/3083
center:(6.812585, 52.404465)
Loading:803/3083
Loading:804/3083
center:(6.817915, 52.284135)
Loading:805/3083
center:(6.756055, 52.22851)
Loading:806/3083
center:(6.823905, 52.389340000000004)
Loading:807/3083
Loading:808/3083
center:(4.83404, 51.564285)
Loading:809/3083
Loading:810/3083
center:(4.721895, 51.600224999999995)
Loading:811/3083
Loading:812/3083
center:(4.642135, 51.57465)
Loading:813/3083
center:(4.854965, 51.648335)
Loading:814/3083
center:(4.78828, 51.571875000000006)
Loading:815/3083
center:(4.756265, 51.579435000000004)
Loading:

...

Loading:2170/3083
Loading:2171/3083
center:(5.842689999999999, 53.201745)
Loading:2172/3083
center:(5.7436799999999995, 53.200375)
Loading:2173/3083
Loading:2174/3083
center:(5.781784999999999, 53.186655)
Loading:2175/3083
center:(6.111935, 53.112785)
Loading:2176/3083
center:(5.926055, 53.14013)
Loading:2177/3083
center:(5.77206, 53.200125)
Loading:2178/3083
center:(6.12125, 53.10955)
Loading:2179/3083
center:(5.992485, 53.191055)
Loading:2180/3083
center:(5.768825, 52.948660000000004)
Loading:2181/3083
Loading:2182/3083
center:(5.817125, 53.198485)
Loading:2183/3083
Loading:2184/3083
center:(5.7930150000000005, 53.190920000000006)
Loading:2185/3083
center:(6.149865, 53.180144999999996)
Loading:2186/3083
center:(5.9961649999999995, 53.325824999999995)
Loading:2187/3083
center:(5.9846, 52.895709999999994)
Loading:2188/3083
Loading:2189/3083
center:(5.79931, 53.18297)
Loading:2190/3083
center:(5.817125, 53.198485)
Loading:2191/3083
Loading:2192/3083
center:(5.7913499999999996, 53.17285)

Loading:2370/3083
center:(6.523944999999999, 53.23146)
Loading:2371/3083
center:(6.504945, 53.300425000000004)
Loading:2372/3083
center:(6.39135, 53.142395)
Loading:2373/3083
center:(6.54916, 53.232945)
Loading:2374/3083
center:(6.61058, 53.16654)
Loading:2375/3083
Loading:2376/3083
Loading:2377/3083
center:(6.869775000000001, 53.106944999999996)
Loading:2378/3083
center:(6.532325, 53.22919)
Loading:2379/3083
center:(6.57375, 53.226295)
Loading:2380/3083
Loading:2381/3083
center:(5.24249, 51.806354999999996)
Loading:2382/3083
center:(5.52487, 51.883750000000006)
Loading:2383/3083
Loading:2384/3083
center:(5.33394, 51.692045)
Loading:2385/3083
Loading:2386/3083
Loading:2387/3083
center:(5.31384, 51.690865)
Loading:2388/3083
Loading:2389/3083
Loading:2390/3083
center:(5.306165, 51.697815000000006)
Loading:2391/3083
center:(5.086435, 51.57085)
Loading:2392/3083
Loading:2393/3083
Loading:2394/3083
center:(5.06829, 51.526529999999994)
Loading:2395/3083
center:(5.0841899999999995, 51.55691)


Loading:2566/3083
Loading:2567/3083
Loading:2568/3083
center:(6.109875, 52.023595)
Loading:2569/3083
center:(6.094875, 52.04822)
Loading:2570/3083
Loading:2571/3083
center:(5.742955, 52.09066)
Loading:2572/3083
Loading:2573/3083
Loading:2574/3083
center:(6.109875, 52.023595)
Loading:2575/3083
center:(5.613035, 52.07794)
Loading:2576/3083
center:(5.537105, 52.035740000000004)
Loading:2577/3083
Loading:2578/3083
center:(5.521325, 52.025890000000004)
Loading:2579/3083
center:(5.567819999999999, 52.019235)
Loading:2580/3083
center:(5.537105, 52.035740000000004)
Loading:2581/3083
center:(5.863015, 51.955185)
Loading:2582/3083
Loading:2583/3083
center:(5.64583, 52.033335)
Loading:2584/3083
center:(5.64583, 52.033335)
Loading:2585/3083
center:(5.557494999999999, 52.025745)
Loading:2586/3083
center:(5.63047, 52.02516)
Loading:2587/3083
Loading:2588/3083
center:(5.557494999999999, 52.025745)
Loading:2589/3083
center:(5.661885, 52.03752)
Loading:2590/3083
center:(6.143915, 51.92142)
Loading:2591

Loading:2758/3083
center:(5.91542, 51.47834)
Loading:2759/3083
center:(6.1919, 51.36004)
Loading:2760/3083
center:(6.1512, 51.358845)
Loading:2761/3083
center:(6.045999999999999, 51.533154999999994)
Loading:2762/3083
center:(5.96084, 51.578514999999996)
Loading:2763/3083
Loading:2764/3083
center:(5.961155, 51.195305000000005)
Loading:2765/3083
center:(5.910235, 51.3465)
Loading:2766/3083
center:(5.989520000000001, 51.32389)
Loading:2767/3083
Loading:2768/3083
center:(6.099075, 51.47457)
Loading:2769/3083
center:(6.174995, 51.359345)
Loading:2770/3083
center:(6.099075, 51.47457)
Loading:2771/3083
Loading:2772/3083
center:(5.96786, 51.307975)
Loading:2773/3083
center:(6.050215, 51.472495)
Loading:2774/3083
center:(6.159075, 51.399879999999996)
Loading:2775/3083
center:(6.189525, 51.384785)
Loading:2776/3083
center:(5.989520000000001, 51.32389)
Loading:2777/3083
Loading:2778/3083
Loading:2779/3083
center:(5.6444600000000005, 51.478165000000004)
Loading:2780/3083
Loading:2781/3083
Loading:

Loading:2946/3083
Loading:2947/3083
Loading:2948/3083
center:(6.34532, 52.703855)
Loading:2949/3083
center:(6.60343, 52.84475)
Loading:2950/3083
Loading:2951/3083
center:(6.41772, 52.900905)
Loading:2952/3083
center:(6.195075, 52.70891)
Loading:2953/3083
Loading:2954/3083
Loading:2955/3083
Loading:2956/3083
center:(6.41772, 52.900905)
Loading:2957/3083
center:(7.0359750000000005, 52.877395)
Loading:2958/3083
center:(6.873825, 52.794785)
Loading:2959/3083
center:(6.98182, 52.922245000000004)
Loading:2960/3083
Loading:2961/3083
center:(6.849410000000001, 52.78247)
Loading:2962/3083
Loading:2963/3083
center:(6.910030000000001, 52.770075000000006)
Loading:2964/3083
Loading:2965/3083
center:(6.9001850000000005, 52.753755)
Loading:2966/3083
center:(6.96663, 52.98304)
Loading:2967/3083
Loading:2968/3083
Loading:2969/3083
center:(6.98182, 52.922245000000004)
Loading:2970/3083
center:(6.88837, 52.758070000000004)
Loading:2971/3083
center:(6.88837, 52.758070000000004)
Loading:2972/3083
center:(6

In [ ]:
df